# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )


In [3]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

# 1.10 countries with the lowest infant mortality rates
Ans:
1. Convert the Xml file into Element tree which is heirarichal tree structure of element tag.
2. Identify the root of the tree and then iterate through tree to find the relevant tag. 
eg:root.getchildren() gives all children of root element
3.To find countries with lowest infant mortality rates, first iterate through  root and identify the country tag which is subelement of root tag mondial
4.Root tag has many country tag, loop through each country tag and to find the infant mortality tag of each country.
5. To get the text of that mortality tag, findtext('tagname') function is used. The result of each iteration of loop is stored in list of dictionaries.
6. Convert the list into Dataframe using pd.Dataframe('listname') and then sort the dataframe based on infant_mortality values in ascending order.
7.The first 10 rows of DataFrame gives name of countries and  the lowest infant mortality rates.

In [5]:
document = ET.parse( './data/mondial_database.xml' )

In [370]:
import pandas as pd
root=document.getroot()
d=[]
for element in root.iterfind('country'):
    d.append({'name' : element.findtext('name'),'infant_mortality':element.findtext('infant_mortality')})
df=pd.DataFrame(d,columns=['name','infant_mortality'])
df.shape
df.isnull().sum()
df.dropna(inplace=True)
df.sort_values('infant_mortality').head(10)


    

    

,name,infant_mortality
38,Monaco,1.81
30,Romania,10.16
153,Fiji,10.2
69,Brunei,10.48
132,Grenada,10.5
237,Mauritius,10.59
124,Panama,10.7
243,Seychelles,10.77
102,United Arab Emirates,10.92
113,Barbados,10.93


# 10 cities with the largest population

Ans:
1.To identify 10 cities with largest population,identify country tags and each country tag has list of city tag.
2. Each city tag has name tag represent name of the city and list of population tags which represent population of city on specific year. 
3.List of dictionaries creates with city name, its population on each year listed in tag
4. Then convert the list into Data frame. Then filter the population of table in latest year. According to the data in table '2014' is latest one and  store the latest result. 
5. Convert the string dtype of population column into integer using astype(int) function. Then sort population in descending order with argument inplace ='True so the changes will be restored in same table.
6.Change index using reset_index() to list the index in ascending order. 
7.df.head(10) fives 10 cities with the largestpopulation

In [8]:
pop_lis=[]
for country in root.iter('country'):
    for city in country.iter('city'):
        name=city.findtext('name')
        lists=list(city.iter('population'))
        for e in lists:
            pop_lis.append({'city_name':name,'year':(e.get('year')),'populations':(e.text)})
pop_df=pd.DataFrame(pop_lis)
pop_df.head(50)
pop_df.sort_values('year',ascending=False,inplace=True)
pop_year_df=pop_df[pop_df.year=='2014']
pop_year_df=pop_year_df.reset_index(range(158))
pop_year_df=pop_year_df.drop('index',1)
pop_year_df['populations']=pop_year_df['populations'].astype(int)
pop_year_df.sort_values('populations',ascending=False,inplace=True)
pop_year_df.head(10)


           

    

,city_name,populations,year
13,Bogotá,7776845,2014
32,Yangon,5209541,2014
101,Abidjan,4395243,2014
39,Medellín,2441123,2014
145,Toshkent,2352900,2014
43,Cali,2344734,2014
23,Conakry,1667864,2014
5,Kampala,1516210,2014
65,Almaty,1507737,2014
100,Mandalay,1225133,2014


#  10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

Ans:
Two ways,it can be done
First Ans
1. Root element has list of country subelements. Each country has list of population tag which represent population on specfied years.
2.Each country has more than one ethnic group and each group represent the name and percentage of their group in that country.
3. Collect all this data in a list of dictionaries using for loop and convert them into Dataframe and result of data frame has following coulumns:
        Country
        year
        population
        ethnicgroup
        percent
4.Group the table so that resulted table will be group of population and percentage of ethnic group of each country based on year. 
5.Convert the data type of population and percent into numeric and then find population of ethnic group by mulitplying percent and population columns and divide the resut by 100.
6.Filter the table for recent year ie 2014 and group the table based on ethnic group and sum the population of each ethnic group and store the result and reset the index of the result.
7. Then sort the ethnicpopulation column in descending column and to find ethnic group has largest population store ethnicgroup and ethnpop column in new table name and to display 10countries with largest ethnic population df.head(10) is used.

In [111]:
d=[]
for country in root.iter('country'):
    tag_lists=list(country.iter('population'))
    for e in tag_lists:
        ethn_list=list(country.iter('ethnicgroup'))
        for i in ethn_list:
            d.append({'country':country.findtext('name'),'year':e.get('year'),'populations':e.text,'ethnicgroup':i.text,'percent':i.get('percentage')})
            
df=pd.DataFrame(d)
df1=df.groupby(['country','ethnicgroup','percent','populations','year']).size()
ehn_df=pd.DataFrame(df1)   
ehn_df=ehn_df.reset_index()
ehn_df=ehn_df.sort_values('year',ascending=False).drop(0,1)
ehn_df['percent']=pd.to_numeric(ehn_df['percent'])
ehn_df['populations']=ehn_df['populations'].astype(int)
ehn_df['ethnpop']=(ehn_df['percent']*ehn_df['populations'])/100
ehn_2014_df=ehn_df[ehn_df.year=='2014']
ehn_2014_df=ehn_2014_df.groupby(['ethnicgroup']).sum()
ehn_2014_df=ehn_2014_df.reset_index()
ehn_2014_df=ehn_2014_df.sort_values('ethnpop',ascending=False)
ehn_2014_df=ehn_2014_df[['ethnicgroup','ethnpop']]
ehn_2014_df=ehn_2014_df.reset_index()
ehn_2014_df.drop('index',1).head(10)

    


,ethnicgroup,ethnpop
0,European,8.358005e+08
1,Russian,2.387651e+08
2,Javanese,2.269120e+08
3,Mulatto,1.721095e+08
4,African,1.119410e+08
5,Arab-Berber,7.647152e+07
6,Burman,7.564417e+07
7,Mestizo,7.325743e+07
8,Sundanese,7.059485e+07
9,Malay,3.781867e+07


Optimal Answer:
1. Build two data frame one is population data frame which has name, year and population of each country
2. Other one is ethnic data frame which has country name, ethnic group and its percentage in ach country.
3. Fileter population data frame to display the recent population ie 2014.
4.Merge these two table on  column'country name' and how='left' which sililar to left join. 
5.Then new colum 'ethnpop' is added and values of this coulumn computed by (population*percent)/100 
6.Group each ethnic group and find sum of popualtion on each group and store the result.
7. Reset index, so that resul table index will be in order.
8.To display 10 ethnic group with largest opulation, the dataframe should have two column 'ethnicgroup' and 'ethnpop' and df.head(10) will display the result. 

In [149]:
d=[]
b=[]
for country in root.iter('country'):
    
    tag_lists=list(country.iter('population'))
    for e in tag_lists:
        d.append({'country':country.findtext('name'),'year':e.get('year'),'population':e.text})
        ethn_list=list(country.iter('ethnicgroup'))
    for i in ethn_list:
            b.append({'country':country.findtext('name'),'ethnicgroup':i.text,'percent':i.get('percentage')})
popul_df=pd.DataFrame(d)
ethgp_df=pd.DataFrame(b)
popul_df=popul_df.sort_values('year',ascending=False)
popul_df['year']=popul_df['year'].astype(int)
popul_df=popul_df[popul_df.year==2014]
merged_df=pd.merge(popul_df,ethgp_df,on='country',how='left')
merged_df['population']=merged_df['population'].astype(int)
merged_df['percent']=pd.to_numeric(merged_df['percent'])
merged_df['overallpop']=(merged_df['population']*merged_df['percent'])/100
merged_df=merged_df.groupby('ethnicgroup').sum()
merged_df=merged_df.reset_index()
merged_df=merged_df[['ethnicgroup','overallpop']]
merged_df.sort_values('overallpop',ascending=False).head(10)



,ethnicgroup,overallpop
23,European,8.373558e+08
55,Russian,2.394448e+08
31,Javanese,2.269120e+08
49,Mulatto,1.731982e+08
1,African,1.122521e+08
46,Mestizo,7.776800e+07
5,Arab-Berber,7.647152e+07
16,Burman,7.643186e+07
62,Sundanese,7.059485e+07
44,Malay,3.781867e+07


# name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [356]:
root.getchildren()[-1].getchildren()

<Element 'airport' at 0x000002362CEA08B8>

# a)Longest River
Sol:
1. Root element has list of river subelement  and each river have name,length as its subelement or children. Lake element has attribute country_code.
2. iterate the root element to identify river and its sublement and store it in list of dictionaries.
3. Convert the list into DataFrame.
4. Similarly, build country dataframe with country name and its code.
5. Sort the river DataFrame in descending order of length value and then merge river_df and country_df on='code' and perform left join. 
6. The result of merge  will have river_name, length, country_code and country name as its column
The first row will give the longest river.

In [354]:

river_list=[]
country_list=[]
for river in root.iter('river'):
    river_list.append({'name':river.findtext('name'),'length':river.findtext('length'),'code':river.get('country')})
for country in root.iter('country'):
    country_list.append({'code': country.get('car_code'),'name':country.findtext('name')})
river_df=pd.DataFrame(river_list)
country_df=pd.DataFrame(country_list)
river_df['length']=pd.to_numeric(river_df['length'])
river_country_df=pd.merge(river_df,country_df,on='code',how='left')
river_country_df.sort_values('length',ascending=False,inplace=True)
river_country_df.head(1).fillna('Colombia Brazil Peru')


,code,length,name_x,name_y
174,CO BR PE,6448.0,Amazonas,Colombia Brazil Peru


# b)Largest Lake
Sol:
1. Root element has list of lake subelement  and each lake has name,length as its subelement or children. Each Lake element has attribute country_code.
2. iterate the root element to identify lake and its sublement and store it in list of dictionaries.
3. Convert the list into DataFrame.
4. Similarly, build country dataframe with country name and its code.
5. Sort the lake DataFrame in descending order of area and then merge lake_df and country_df on='code' and perform left join. 
6. The result df will have lake_name, length, country_code and country name as its column.
The first row will give the largest lake.

In [353]:
lake_list=[]
for lake in root.iter('lake'):
     lake_list.append({'lake_name':lake.findtext('name'),'code':lake.get('country'),'area':lake.findtext('area')})
lake_df=pd.DataFrame(lake_list,columns=['lake_name','code','area'])
lake_df['area']=pd.to_numeric(lake_df['area'])
lake_df=lake_df.sort_values('area',ascending=False)
lake_country_df=pd.merge(lake_df,country_df,on='code',how='left')
lake_country_df.head(1).fillna('Russia Kazakhsta Iran Turkmenistan')

   

,lake_name,code,area,name
0,Caspian Sea,R AZ KAZ IR TM,386400.0,Russia Kazakhsta Iran Turkmenistan


# c)airport at highest elevation
1. Root element has list of airport subelements  and each airport has name and length as its subelement or children. Each airport element has attribute country_code.
2. iterate the root element to identify airport and its sublement and store it in list of dictionaries.
3. Convert this list into DataFrame.
4. Similarly,build country dataframe with country name and its code.
5. Sort the airport DataFrame in descending order of elevation value and then merge airport_df and country_df on='code' and perform left join. 
6. Then result will have airport_name, elevation, country_code and country name as its column.
The first row will give the airport with highest elevation.

In [357]:
root.getchildren()[-1].getchildren()

[<Element 'name' at 0x000002362CEA0908>,
 <Element 'latitude' at 0x000002362CEA0958>,
 <Element 'longitude' at 0x000002362CEA09A8>,
 <Element 'elevation' at 0x000002362CEA09F8>,
 <Element 'gmtOffset' at 0x000002362CEA0A48>]

In [366]:
airport_list=[]
for airport in root.iter('airport'):
    airport_list.append({'airport_name':airport.findtext('name'),'elevation': airport.findtext('elevation'),'code':airport.get('country')})
airport_df=pd.DataFrame(airport_list)
airport_country_df=pd.merge(airport_df,country_df,on='code',how='left')
airport_country_df.sort_values('elevation',ascending=False,inplace=True)
airport_country_df.head(1)

,airport_name,code,elevation,name
536,Mashhad,IR,995,Iran
